In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from scipy.stats import norm, bernoulli
import json
import warnings; warnings.simplefilter('ignore')
from tqdm.auto import tqdm
from plotting_utils import plot_naive_coverage, plot_eff_sample_size, plot_coverage, plot_intervals
from utils import train_tree, tree_predict, opt_mean_tuning, mean_ESS

## Load data

In [ ]:
data = pd.read_csv('datasets/bias_dataset.csv')
data = data.sample(frac=1).reset_index(drop=True) # shuffle data
leaning = 'left' # 'left' or 'right'
Yhat_string = data["label_gpt4o"].to_numpy()
Y_string = data["bias_text"].to_numpy()
confidence = data["confidence_in_prediction_gpt-4o"].to_numpy()
nan_indices = list(np.where(pd.isna(confidence))[0]) + list(np.where(pd.isna(Yhat_string))[0])
good_indices = list(set(range(len(data))) - set(nan_indices))
confidence = confidence[good_indices]
Yhat_string = Yhat_string[good_indices]
Y_string = Y_string[good_indices]
n = len(Yhat_string)
if leaning=='left':
    dict = {"A" : 1, "B" : 0, "C" : 0, "left" : 1, "center": 0, "right": 0}
elif leaning=='right':
    dict = {"A" : 0, "B" : 0, "C" : 1, "left" : 0, "center": 0, "right": 1}
Yhat = np.array([dict[Yhat_string[i]] for i in range(n)])
Y = np.array([dict[Y_string[i]] for i in range(n)])
confidence = confidence.reshape(len(confidence),1)

## Effective sample size and coverage

In [ ]:
alpha = 0.1  # desired error level for confidence interval
fracs_human = np.linspace(0.2, 0.6, 10)  # overall sampling budget
num_trials = 100

true_prevalence = np.mean(Y)

num_methods = 4
temp_df = pd.DataFrame({
    "lb": np.zeros(num_methods),
    "ub": np.zeros(num_methods),
    "interval width": np.zeros(num_methods),
    "coverage": np.zeros(num_methods),
    "estimator": [""] * num_methods,
    "$n_{\mathrm{human}}$": np.zeros(num_methods),
    "$n_{\mathrm{effective}}$": np.zeros(num_methods)
})


burnin_steps = 50  # we collect the first burnin_steps points to initialize sampling rule
retrain_steps = 50  # every retrain_steps we retrain mapping from confidence to sampling probability
tau = 0.1  # parameter for mixing with uniform sampling for increased stability

results = []

for j in tqdm(range(len(fracs_human))):
    frac_human = fracs_human[j]
    frac_human_adjusted = (frac_human*n - burnin_steps)/(n - burnin_steps) # remove burnin_steps samples from available budget
    
    for i in tqdm(range(num_trials)):
        tree = train_tree(confidence[:burnin_steps], ((Y - Yhat)[:burnin_steps])**2)
        uncertainties = np.sqrt(tree_predict(tree, confidence))
        avg_uncertainty = np.mean(uncertainties)
        weights_active = np.zeros(n)
        weights_active[:burnin_steps] = 1
        sampling_ratio = np.zeros(n)
        
        for t in range(burnin_steps, n):

            if ((t-burnin_steps) % retrain_steps == 0):
                obs_inds = np.where(weights_active)
                tree = train_tree(confidence[obs_inds], ((Y - Yhat)[obs_inds])**2)
                uncertainties = np.sqrt(tree_predict(tree, confidence))
                avg_uncertainty = np.mean(uncertainties)

            sampling_prob = uncertainties[t]/avg_uncertainty*frac_human_adjusted
            sampling_prob = np.clip((1-tau)*sampling_prob + tau*frac_human_adjusted, 0, 1)
            sampling_ratio[t] = (1-sampling_prob)/sampling_prob

            weights_active[t] = bernoulli.rvs(sampling_prob)/sampling_prob

        lam = opt_mean_tuning(Y, Yhat, weights_active, sampling_ratio)
        pointest = np.mean(lam*Yhat + (Y-lam*Yhat)*weights_active)
        varhat = np.var(lam*Yhat + (Y-lam*Yhat)*weights_active)
        l = pointest - norm.ppf(1-alpha/2)*np.sqrt(varhat/n)
        u = pointest + norm.ppf(1-alpha/2)*np.sqrt(varhat/n)
        coverage = (true_prevalence >= l)*(true_prevalence <= u)
        temp_df.loc[0] = l, u, u-l, coverage, "confidence-driven", int(n*frac_human), mean_ESS(Y, varhat)
        
        xi_unif = bernoulli.rvs([frac_human] * n)
        pointest = np.mean(Yhat + (Y-Yhat)*xi_unif/frac_human)
        varhat = np.var(Yhat + (Y-Yhat)*xi_unif/frac_human)
        l = pointest - norm.ppf(1-alpha/2)*np.sqrt(varhat/n)
        u = pointest + norm.ppf(1-alpha/2)*np.sqrt(varhat/n)
        coverage = (true_prevalence >= l)*(true_prevalence <= u)
        temp_df.loc[1] = l, u, u-l, coverage, "human + LLM (non-adaptive)", int(n*frac_human), mean_ESS(Y, varhat)

        pointest = np.mean(Y*xi_unif/frac_human)
        varhat = np.var(Y*xi_unif/frac_human)
        l = pointest - norm.ppf(1-alpha/2)*np.sqrt(varhat/n)
        u = pointest + norm.ppf(1-alpha/2)*np.sqrt(varhat/n)
        coverage = (true_prevalence >= l)*(true_prevalence <= u)
        temp_df.loc[2] = l, u, u-l, coverage, "human only", int(n*frac_human), mean_ESS(Y, varhat)

        LLM_only_sample = np.random.choice(n, n, replace=True)
        pointest = np.mean(Yhat[LLM_only_sample])
        varhat = np.var(Yhat[LLM_only_sample])
        l = pointest - norm.ppf(1-alpha/2)*np.sqrt(varhat/n)
        u = pointest + norm.ppf(1-alpha/2)*np.sqrt(varhat/n)
        coverage = (true_prevalence >= l)*(true_prevalence <= u)
        temp_df.loc[3] = l, u, u-l, coverage, "LLM only", int(n*frac_human), 0
        
        results += [temp_df.copy()]
df = pd.concat(results,ignore_index=True)
df["coverage"] = df["coverage"].astype(bool)

In [ ]:
plot_eff_sample_size(df, "plots/" + leaning + "_leaning_ESS.pdf")

In [ ]:
plot_coverage(df, alpha, "plots/" + leaning + "_leaning_coverage.pdf")

In [ ]:
plot_intervals(df, true_prevalence, num_trials, "prevalence $p_{\mathrm{" + leaning + "}}$", "plots/" + leaning + "_leaning_intervals.pdf", n_ind=0)

## Experiment at fixed $n_{\text{human}}$

In [ ]:
alpha = 0.1  # desired error level for confidence interval
num_trials = 100
nhuman = 500
frac_human = nhuman/n

true_prevalence = np.mean(Y)

num_methods = 3
temp_df = pd.DataFrame({
    "estimator": [""] * num_methods,
    "coverage": np.zeros(num_methods),
    "$n_{\mathrm{effective}}$": np.zeros(num_methods)
})


burnin_steps = 50  # we collect the first burnin_steps points to initialize sampling rule
retrain_steps = 50  # every retrain_steps we retrain mapping from confidence to sampling probability
tau = 0.1  # parameter for mixing with uniform sampling for increased stability

results = []

frac_human_adjusted = (nhuman - burnin_steps)/(n - burnin_steps) # remove burnin_steps samples from available budget

for i in tqdm(range(num_trials)):
    tree = train_tree(confidence[:burnin_steps], ((Y - Yhat)[:burnin_steps])**2)
    uncertainties = np.sqrt(tree_predict(tree, confidence))
    avg_uncertainty = np.mean(uncertainties)
    weights_active = np.zeros(n)
    weights_active[:burnin_steps] = 1
    sampling_ratio = np.zeros(n)
    
    for t in range(burnin_steps, n):

        if ((t-burnin_steps) % retrain_steps == 0):
            obs_inds = np.where(weights_active)
            tree = train_tree(confidence[obs_inds], ((Y - Yhat)[obs_inds])**2)
            uncertainties = np.sqrt(tree_predict(tree, confidence))
            avg_uncertainty = np.mean(uncertainties)

        sampling_prob = uncertainties[t]/avg_uncertainty*frac_human_adjusted
        sampling_prob = np.clip((1-tau)*sampling_prob + tau*frac_human_adjusted, 0, 1)
        sampling_ratio[t] = (1-sampling_prob)/sampling_prob

        weights_active[t] = bernoulli.rvs(sampling_prob)/sampling_prob

    lam = opt_mean_tuning(Y, Yhat, weights_active, sampling_ratio)
    pointest = np.mean(lam*Yhat + (Y-lam*Yhat)*weights_active)
    varhat = np.var(lam*Yhat + (Y-lam*Yhat)*weights_active)
    l = pointest - norm.ppf(1-alpha/2)*np.sqrt(varhat/n)
    u = pointest + norm.ppf(1-alpha/2)*np.sqrt(varhat/n)
    coverage = (true_prevalence >= l)*(true_prevalence <= u)
    temp_df.loc[0] = "confidence-driven", coverage, mean_ESS(Y, varhat)
    
    xi_unif = bernoulli.rvs([frac_human] * n)
    pointest = np.mean(Yhat + (Y-Yhat)*xi_unif/frac_human)
    varhat = np.var(Yhat + (Y-Yhat)*xi_unif/frac_human)
    l = pointest - norm.ppf(1-alpha/2)*np.sqrt(varhat/n)
    u = pointest + norm.ppf(1-alpha/2)*np.sqrt(varhat/n)
    coverage = (true_prevalence >= l)*(true_prevalence <= u)
    temp_df.loc[1] = "human + LLM (non-adaptive)", coverage, mean_ESS(Y, varhat)

    LLM_only_sample = np.random.choice(n, n, replace=True)
    pointest = np.mean(Yhat[LLM_only_sample])
    varhat = np.var(Yhat[LLM_only_sample])
    l = pointest - norm.ppf(1-alpha/2)*np.sqrt(varhat/n)
    u = pointest + norm.ppf(1-alpha/2)*np.sqrt(varhat/n)
    coverage = (true_prevalence >= l)*(true_prevalence <= u)
    temp_df.loc[2] = "LLM only", coverage, 0
    
    results += [temp_df.copy()]
df = pd.concat(results,ignore_index=True)
df["ESS gain"] = df["$n_{\mathrm{effective}}$"]/nhuman - 1
df["coverage"] = df["coverage"].astype(bool)